goal add GCN to the MAPPO

workflow
build roadmap
for custom API, use kimi to create a cheat sheet, from the doc
useGPT to generate code
debug


problem:

what is observation of vmas? can we build graph from it?

can we use crazy file?

graph will be changed? can 

how to save graph in tensordict?

# 1 in vmas

In [30]:
# Torch
import torch

from tensordict.nn import TensorDictModule
from tensordict.nn.distributions import NormalParamExtractor

# Tensordict modules
from torch import multiprocessing

# Data collection
from torchrl.collectors import SyncDataCollector
from torchrl.data.replay_buffers import ReplayBuffer
from torchrl.data.replay_buffers.samplers import SamplerWithoutReplacement
from torchrl.data.replay_buffers.storages import LazyTensorStorage

# Env
from torchrl.envs import RewardSum, TransformedEnv
from torchrl.envs.libs.vmas import VmasEnv
from torchrl.envs.utils import check_env_specs

# Multi-agent network
from torchrl.modules import MultiAgentMLP, ProbabilisticActor, TanhNormal

# Loss
from torchrl.objectives import ClipPPOLoss, ValueEstimators

# Utils
torch.manual_seed(0)
from matplotlib import pyplot as plt
from tqdm import tqdm




# Devices
is_fork = multiprocessing.get_start_method() == "fork"
device = (
    torch.device(0)
    if torch.cuda.is_available() and not is_fork
    else torch.device("cpu")
)
vmas_device = device  # The device where the simulator is run (VMAS can run on GPU)

# Sampling
frames_per_batch = 6_000  # Number of team frames collected per training iteration
n_iters = 10  # Number of sampling and training iterations
total_frames = frames_per_batch * n_iters

# Training
num_epochs = 30  # Number of optimization steps per training iteration
minibatch_size = 400  # Size of the mini-batches in each optimization step
lr = 3e-4  # Learning rate
max_grad_norm = 1.0  # Maximum norm for the gradients

# PPO
clip_epsilon = 0.2  # clip value for PPO loss
gamma = 0.9  # discount factor
lmbda = 0.9  # lambda for generalised advantage estimation
entropy_eps = 1e-4  # coefficient of the entropy term in the PPO loss

max_steps = 100  # Episode steps before done
num_vmas_envs = (
    frames_per_batch // max_steps
)  # Number of vectorized envs. frames_per_batch should be divisible by this number
scenario_name = "navigation"
n_agents = 3

env = VmasEnv(
    scenario=scenario_name,
    num_envs=num_vmas_envs,
    continuous_actions=True,  # VMAS supports both continuous and discrete actions
    max_steps=max_steps,
    device=vmas_device,
    # Scenario kwargs
    n_agents=n_agents,  # These are custom kwargs that change for each VMAS scenario, see the VMAS repo to know more.
)

env = TransformedEnv(
    env,
    RewardSum(in_keys=[env.reward_key], out_keys=[("agents", "episode_reward")]),
)

check_env_specs(env)

print(env.reset())


2024-03-15 08:51:33,603 [torchrl][INFO] check_env_specs succeeded!


TensorDict(
    fields={
        agents: TensorDict(
            fields={
                episode_reward: Tensor(shape=torch.Size([60, 3, 1]), device=cuda:0, dtype=torch.float32, is_shared=True),
                info: TensorDict(
                    fields={
                        agent_collisions: Tensor(shape=torch.Size([60, 3, 1]), device=cuda:0, dtype=torch.float32, is_shared=True),
                        final_rew: Tensor(shape=torch.Size([60, 3, 1]), device=cuda:0, dtype=torch.float32, is_shared=True),
                        pos_rew: Tensor(shape=torch.Size([60, 3, 1]), device=cuda:0, dtype=torch.float32, is_shared=True)},
                    batch_size=torch.Size([60, 3]),
                    device=cuda:0,
                    is_shared=True),
                observation: Tensor(shape=torch.Size([60, 3, 18]), device=cuda:0, dtype=torch.float32, is_shared=True)},
            batch_size=torch.Size([60, 3]),
            device=cuda:0,
            is_shared=True),
        done: T

## what is observation of vmas?
18 is the dim of observation


```
return torch.cat(
            [
                agent.state.pos,
                agent.state.vel,
            ]
            + goal_poses
            + (
                [agent.sensors[0]._max_range - agent.sensors[0].measure()]
                if self.collisions
                else []
            ),
            dim=-1,
        )
```

The observation for the agent in this scenario is composed of several parts, as defined in the `observation` method of the `Scenario` class:

1. `agent.state.pos`: The current position of the agent in the environment. This is a vector that typically includes the x and y coordinates (and possibly z in 3D environments), which is essential for the agent to understand its location in the world.

2. `agent.state.vel`: The velocity of the agent, which provides information about its movement speed and direction. This can be useful for predicting future positions and for reacting to dynamic changes in the environment.

3. `goal_poses`: A list of vectors representing the positions of the goals relative to the agent's current position. If `self.observe_all_goals` is `True`, this list includes the goal positions for all agents; otherwise, it only includes the goal position for the specific agent. This information helps the agent to know where its goal is and how to navigate towards it.

4. `agent.sensors[0]._max_range - agent.sensors[0].measure()`: If collisions are enabled (`self.collisions` is `True`), this part of the observation represents the distance measurement from the agent's lidar sensor. The lidar sensor provides a range of measurements, and this value indicates the difference between the maximum range of the sensor and the actual measured distance. This can be useful for detecting obstacles and other agents in the environment.

The shape of the observation is determined by concatenating all these parts using `torch.cat`. The resulting shape will be a 1D tensor with a length that corresponds to the number of elements in each part. For example, if the position is a 2D vector (x, y), the velocity is a 2D vector (dx, dy), and there are `n` goals, the observation shape will be `2 (position) + 2 (velocity) + n (goal positions) + 1 (lidar measurement if collisions)`.

The meaning of the observation is to provide the agent with a comprehensive view of its state and the surrounding environment, enabling it to make informed actions that will help it achieve its goal(s) while avoiding collisions and navigating effectively.

In [3]:
print(env.observation_spec['agents']['observation'])

UnboundedContinuousTensorSpec(
    shape=torch.Size([60, 3, 18]),
    space=None,
    device=cuda:0,
    dtype=torch.float32,
    domain=continuous)


In [4]:
# Reset the environment to get the initial state
initial_state = env.reset()

# Access the observations for agents
observations = initial_state['agents']['observation']

# Print the observations tensor
print(observations[0,0])


tensor([-0.7314,  0.2468,  0.0000,  0.0000, -0.4608,  0.7366,  0.0000,  0.0520,
         0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000], device='cuda:0')


In [5]:
print("obs_spec:", env.full_observation_spec)

obs_spec: CompositeSpec(
    agents: CompositeSpec(
        observation: UnboundedContinuousTensorSpec(
            shape=torch.Size([60, 3, 18]),
            space=None,
            device=cuda:0,
            dtype=torch.float32,
            domain=continuous),
        info: CompositeSpec(
            pos_rew: UnboundedContinuousTensorSpec(
                shape=torch.Size([60, 3, 1]),
                space=None,
                device=cuda:0,
                dtype=torch.float32,
                domain=continuous),
            final_rew: UnboundedContinuousTensorSpec(
                shape=torch.Size([60, 3, 1]),
                space=None,
                device=cuda:0,
                dtype=torch.float32,
                domain=continuous),
            agent_collisions: UnboundedContinuousTensorSpec(
                shape=torch.Size([60, 3, 1]),
                space=None,
                device=cuda:0,
                dtype=torch.float32,
                domain=continuous), device=

build connected graph for all agent, and put it into the tensordict

---
how to update tensordict

dummy_agent_collisions = torch.randn((60, 3, 1), device='cuda:0')
obs_tensor_dict.agents.info.agent_collisions = dummy_agent_collisions

Based on the information provided, it seems that `TensorDict` is a custom data structure that is designed to handle key-value pairs where the values are tensors. This structure appears to have methods and properties that are similar to PyTorch tensors, allowing for operations like indexing, stacking, concatenating, reshaping, and device management.

Here's a cheatsheet for some of the key features and operations you can perform with `TensorDict`:

Creating a TensorDict
```python
from tensordict import TensorDict
import torch

# Create a TensorDict with initial tensors
data = TensorDict({
    "key 1": torch.ones(3, 4, 5),
    "key 2": torch.zeros(3, 4, 5, dtype=torch.bool),
}, batch_size=[3, 4])
```
Adding a New Key-Value Pair
```python
# Add a new tensor to the TensorDict
data["new_key"] = torch.randn(3, 4)
```
 most accurate information.

### transform the environment

an naive idea is to add index_id and batch to the top level on env, but got `User
RuntimeError: batch dimension mismatch, got self.batch_size=torch.Size([60]) and value.shape=torch.Size([2, 6]).`

In [38]:


import torch
from torchrl.envs import Transform
from tensordict import TensorDictBase

class AddGraphDataTransform(Transform):
    def __init__(self, num_agents, num_envs, device):
        super().__init__()
        self.num_agents = num_agents
        self.num_envs = num_envs
        self.device = device
        self.edge_index, self.batch = self.create_graph_data()

    def create_graph_data(self):
        # Assuming each environment has a fully connected graph of 3 agents
        edges = [(i, j) for i in range(self.num_agents) for j in range(self.num_agents) if i != j]
        edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

        # Flatten or expand edge_index to match the [60, 3, ...] structure
        # This is a placeholder operation and may not be meaningful for actual graph convolutions
        edge_index = edge_index.view(1, 2, -1).expand(self.num_envs, -1, -1)

        # Expand the batch tensor similarly
        # Placeholder operation
        batch = torch.arange(self.num_envs).unsqueeze(-1).expand(-1, self.num_agents).flatten()
        batch = batch.view(1, 1, -1).expand(-1, 3, -1)

        return edge_index.to(self.device), batch.to(self.device)


    def _apply_transform(self, tensordict: TensorDictBase) -> TensorDictBase:
        # Add edge_index and batch tensors to the tensordict
        tensordict.set("edge_index", self.edge_index)
        tensordict.set("batch", self.batch)
        return tensordict

    def _call(self, tensordict: TensorDictBase) -> TensorDictBase:
        return self._apply_transform(tensordict)
    def _reset(
        self, tensordict: TensorDictBase, tensordict_reset: TensorDictBase
    ) -> TensorDictBase:
        # Ensure the transform is applied during environment resets
        return self._apply_transform(tensordict_reset)

# Example usage
# Assuming num_agents, num_envs, and device are defined
graph_transform = AddGraphDataTransform(num_agents, num_envs, device)

# When creating your transformed environment
env = TransformedEnv(env, graph_transform)
print(env.reset())


RuntimeError: batch dimension mismatch, got self.batch_size=torch.Size([60]) and value.shape=torch.Size([2, 6]).

use transform, instead of changing the env directly



In [8]:
import torch

num_agents = 3

# Create a fully connected graph for 3 agents
edges = [(i, j) for i in range(num_agents) for j in range(num_agents) if i != j]

# Convert edge list to edge index tensor
edge_index = torch.tensor(edges, dtype=torch.long).t().contiguous()

# Expand to match the number of environments
num_envs = 60
edge_index = edge_index.unsqueeze(0).repeat(num_envs, 1, 1).to('cuda:0')
print(edge_index.shape)  # Output: torch.Size([60, 2, 6])
# Edge index tensor is now of shape [60, 2, num_edges] and on cuda:0 device




# Assuming edge_index tensor is already defined and on the correct device
# Example shape: [60, 2, num_edges]

# Create a TensorSpec for the edge index
edge_index_spec = UnboundedContinuousTensorSpec(
    shape=edge_index.shape,
    device='cuda:0',
    dtype=torch.long,
    domain='discrete'
)

# Add the edge index spec to the observation_spec at a top-level key
env.observation_spec['agent_graph'] = edge_index_spec


# Note: This will replace any existing 'agent_graph' spec.


torch.Size([60, 2, 6])


In [9]:
n_rollout_steps = 5
rollout = env.rollout(n_rollout_steps)
print("rollout of three steps:", rollout)
print("Shape of the rollout TensorDict:", rollout.batch_size)

rollout of three steps: TensorDict(
    fields={
        agents: TensorDict(
            fields={
                action: Tensor(shape=torch.Size([60, 5, 3, 2]), device=cuda:0, dtype=torch.float32, is_shared=True),
                episode_reward: Tensor(shape=torch.Size([60, 5, 3, 1]), device=cuda:0, dtype=torch.float32, is_shared=True),
                info: TensorDict(
                    fields={
                        agent_collisions: Tensor(shape=torch.Size([60, 5, 3, 1]), device=cuda:0, dtype=torch.float32, is_shared=True),
                        final_rew: Tensor(shape=torch.Size([60, 5, 3, 1]), device=cuda:0, dtype=torch.float32, is_shared=True),
                        pos_rew: Tensor(shape=torch.Size([60, 5, 3, 1]), device=cuda:0, dtype=torch.float32, is_shared=True)},
                    batch_size=torch.Size([60, 5, 3]),
                    device=cuda:0,
                    is_shared=True),
                observation: Tensor(shape=torch.Size([60, 5, 3, 18]), device=cud

### GNN usage
GNN can't read vectorized data, which is contradict to the requirement of tensordict

In [29]:
import torch
from torch_geometric.data import Data

# Example: Creating two fully connected graphs with 3 nodes each
def create_graph(num_nodes):
    edge_index = torch.combinations(torch.arange(num_nodes), r=2, with_replacement=False)
    edge_index = torch.cat([edge_index, edge_index.flip([1])], dim=0).t().contiguous()
    x = torch.rand((num_nodes, 5))  # Example node features
    return Data(x=x, edge_index=edge_index)

graph1 = create_graph(3)
graph2 = create_graph(3)

from torch_geometric.data import Batch

# Batching the graphs
batched_graph = Batch.from_data_list([graph1, graph2])

from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(5, 10)
        self.conv2 = GCNConv(10, 2)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = torch.relu(x)
        x = self.conv2(x, edge_index)
        return x

model = GCN()

# Forward pass to get node embeddings
node_embeddings = model(batched_graph)
print(node_embeddings)


tensor([[-0.1115, -0.2273],
        [-0.1115, -0.2273],
        [-0.1115, -0.2273],
        [-0.3962, -0.2293],
        [-0.3962, -0.2293],
        [-0.3962, -0.2293]], grad_fn=<AddBackward0>)


# define network

GPT define from scratch, but it is not used thisway, 
refer to the     MultiAgentMLP or MAPPO_new
in that case, GCN might be use

In [24]:
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.data import Batch
# Assume graph_tensor is your graph representation, e.g., an adjacency matrix

class GCNPolicyNetwork(torch.nn.Module):
    def __init__(self, num_features, num_actions, num_agents, device):
        super(GCNPolicyNetwork, self).__init__()
        # GCN layer
        self.gcn = GCNConv(num_features, 128)
        # Further layers
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(128, 256),
            torch.nn.Tanh(),
            torch.nn.Linear(256, 2 * num_actions)
        )

    def forward(self, x, edge_index):
        # Create a batched graph (if each graph in the batch is identical)
        edge_index = Batch.from_data_list([edge_index for _ in range(batch_size)]).edge_index

        # Apply GCN
        x = self.gcn(x, edge_index)
        x = x.view(num_agents, -1)  # Reshape to combine agent features
        # Apply MLP
        x = self.mlp(x)
        return x

# Modify your policy network to use GCNPolicyNetwork
n_features_per_agent = env.observation_spec["agents", "observation"].shape[-1]
n_actions_per_agent = env.action_spec.shape[-1]
num_agents = env.n_agents

policy_net = GCNPolicyNetwork(
    num_features=n_features_per_agent,
    num_actions=n_actions_per_agent,
    num_agents=num_agents,
    device=device
)

# Wrap the GCNPolicyNetwork in a TensorDictModule
policy_module = TensorDictModule(
    policy_net,
    in_keys=[("agents", "observation"), "agent_graph"],  # Include the graph tensor here
    out_keys=[("agents", "loc"), ("agents", "scale")]
)

policy = ProbabilisticActor(
    module=policy_module,
    spec=env.unbatched_action_spec,
    in_keys=[("agents", "loc"), ("agents", "scale")],
    out_keys=[env.action_key],
    distribution_class=TanhNormal,
    distribution_kwargs={
        "min": env.unbatched_action_spec[env.action_key].space.low,
        "max": env.unbatched_action_spec[env.action_key].space.high,
    },
    return_log_prob=True,
    log_prob_key=("agents", "sample_log_prob"),
)  # we'll need the log-prob for the PPO loss



In [16]:
class GCNCriticNetwork(torch.nn.Module):
    def __init__(self, num_features, num_agents, device):
        super(GCNCriticNetwork, self).__init__()
        # GCN layer
        self.gcn = GCNConv(num_features, 128)
        # Further layers
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(128 * num_agents, 256),  # Combine features from all agents
            torch.nn.Tanh(),
            torch.nn.Linear(256, 1)  # Output a single value for the state
        )

    def forward(self, x, edge_index):
        # Apply GCN
        x = self.gcn(x, edge_index)
        x = x.view(1, -1)  # Combine features from all agents
        # Apply MLP
        x = self.mlp(x)
        return x

# Modify your critic network to use GCNCriticNetwork
n_features_per_agent = env.observation_spec["agents", "observation"].shape[-1]
num_agents = env.n_agents

critic_net = GCNCriticNetwork(
    num_features=n_features_per_agent,
    num_agents=num_agents,
    device=device
)

# Wrap the GCNCriticNetwork in a TensorDictModule
critic = TensorDictModule(
    module=critic_net,
    in_keys=[("agents", "observation"), "graph_tensor"],  # Include the graph tensor here
    out_keys=[("agents", "state_value")],
)


In [28]:
import torch
from torch_geometric.nn import GCNConv

class GCNCriticNetwork(torch.nn.Module):
    def __init__(self, num_features, num_agents, device):
        super(GCNCriticNetwork, self).__init__()
        # GCN layer
        self.gcn = GCNConv(num_features, 128)
        # MLP layers for combining GCN output with other observations
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(128 + num_features, 256),  # Assuming additional observation features are concatenated
            torch.nn.Tanh(),
            torch.nn.Linear(256, 1)  # Outputting a single value
        )

    def forward(self, x, edge_index):
        # Apply GCN
        gcn_output = self.gcn(x, edge_index)
        # Combine GCN output with other observations
        combined_input = torch.cat([gcn_output, x], dim=1)
        # Apply MLP
        value = self.mlp(combined_input)
        return value

# Instantiate the network
critic_net = GCNCriticNetwork(
    num_features=env.observation_spec["agents", "observation"].shape[-1],
    num_agents=env.n_agents,
    device=device
)
critic = TensorDictModule(
    module=critic_net,
    in_keys=[("agents", "observation"), "agent_graph"],  # Include the graph tensor here
    out_keys=[("agents", "state_value")],
)
# Continue with the TensorDictModule, etc.
critic_net.to(device)
print("Running critic:", critic(env_state))

ValueError: `MessagePassing.propagate` only supports integer tensors of shape `[2, num_messages]`, `torch_sparse.SparseTensor` or `torch.sparse.Tensor` for argument `edge_index`.

In [25]:
# Move your models to the correct device
policy_net.to(device)
critic_net.to(device)

# Example of running the policy and critic on a batch of data
env_state = env.reset()

# Move the input data to the correct device
# Note: Ensure that env_state is a TensorDict or similar structure that can be moved to a device
env_state = env_state.to(device)

# Run the policy and critic with the data
print("Running policy:", policy(env_state))
print("Running critic:", critic(env_state))


NameError: name 'batch_size' is not defined